In [2]:
from TradingView import Datafeed
Data = Datafeed()
Data.Binance_daily_close(symbol='BTCUSD', date_start_str='2021/01/01', date_end_str='2022/01/01')

you are using nologin method, data you access may be limited


datetime
2021-01-09 08:00:00    39298.747335
2021-01-10 08:00:00    37705.265080
2021-01-11 08:00:00    35435.499061
2021-01-12 08:00:00    34105.017721
2021-01-13 08:00:00    37396.067379
                           ...     
2022-01-04 08:00:00    45871.550000
2022-01-05 08:00:00    43544.170000
2022-01-06 08:00:00    43187.320000
2022-01-07 08:00:00    41603.710000
2022-01-08 08:00:00    41941.350000
Name: close, Length: 365, dtype: float64